## Text Classification - Finetuning 

In [ ]:

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


In [17]:
#Load dataset

from datasets import load_dataset

dataset = load_dataset("copenlu/scientific-exaggeration-detection")
#dataset = load_dataset("glue", "mrpc")

Using custom data configuration copenlu--scientific-exaggeration-detection-9f58052a28a408a8
Found cached dataset json (/home/kyagci/.cache/huggingface/datasets/copenlu___json/copenlu--scientific-exaggeration-detection-9f58052a28a408a8/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# Load tokenizer

from transformers import AutoTokenizer

checkpoint = "roberta-large" 
#checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/kyagci/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /home/kyagci/.cache/h

In [18]:
# Tokenize

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["abstract_conclusion"], 
        examples["press_release_conclusion"],
        #examples["sentence1"], 
        #examples["sentence2"],
        #padding=True,  
        truncation=True,
        #max_length=128, 
        #return_tensors="pt",
        )
    
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/kyagci/.cache/huggingface/datasets/copenlu___json/copenlu--scientific-exaggeration-detection-9f58052a28a408a8/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-ee6fef0654101a71.arrow
Loading cached processed dataset at /home/kyagci/.cache/huggingface/datasets/copenlu___json/copenlu--scientific-exaggeration-detection-9f58052a28a408a8/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-9d0eee0b5e94d5e3.arrow


In [5]:
#tokenized_dataset["train"][0]
#tokenized_dataset["train"][0]['input_ids']

In [19]:
from transformers import DataCollatorWithPadding, DataCollator

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#data_collator = DataCollator(tokenizer)

In [24]:
# Load model

from transformers import AutoModelForMaskedLM, AutoModel, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

loading configuration file config.json from cache at /home/kyagci/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [21]:
#small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(100))
#small_eval_dataset = tokenized_dataset["validation"].shuffle(seed=42).select(range(100))

In [22]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

training_args2 = TrainingArguments("test-trainer")

trainer = Trainer(
    model=model,
    args=training_args2,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    #train_dataset=small_train_dataset,
    #eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: press_release_strength, original_file_id, exaggeration_label, press_release_conclusion, abstract_conclusion, abstract_strength. If press_release_strength, original_file_id, exaggeration_label, press_release_conclusion, abstract_conclusion, abstract_strength are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/home/kyagci/anaconda3/envs/base_env1/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [14]:
# Runs only for "glue", "mrpc"

# import evaluate
# import numpy as np

# predictions = trainer.predict(tokenized_dataset["validation"])
# print(predictions.predictions.shape, predictions.label_ids.shape)

# preds = np.argmax(predictions.predictions, axis=-1)

# metric = evaluate.load("glue", "mrpc")
# metric.compute(predictions=preds, references=predictions.label_ids)